# Assignment 1 - Finetuning YOLO on COCO Traffic images

In [1]:
import torch
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/tobias/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Dataset

For our first project we chose the provided **COCO Dataset**

It is an open dataset for region segmentation, that is hosted as a challenge. As such, **the test set annotations are not available**.

The COCO dataset has 80 object categories, including common objects like cars, bicycles, and animals, as well as more specific categories such as umbrellas, handbags, and sports equipment. For our task we will not need all of these, but luckily with the implementation of ultralytics we can select a subset of these to train on.

Concretely we will group them as:
### Pedestrians
* Person
* Dog
* Cow
* Horse
* Cat

### Vehicle
* Motorcycle
* Bus
* Car
* Truck
* Train

### Cyclist
* Bicycle



TODO: visualize some of the stats of the dataset, as well as some example images with annotations

## Training

For a first attempt, we can use ultralytics' implementation of a training without any augmentations, which we can add later to check and compare results.

TODO: actually run and train the model with parameters setting like in: https://docs.ultralytics.com/modes/train/

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco.yaml", epochs=100, imgsz=640)

## Evaluation

TODO evaluate the model, consult this website.

https://docs.ultralytics.com/modes/val/

In [ ]:
# Load a model
model = YOLO("yolo11n.pt")  # load an official model
model = YOLO("path/to/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

## Video Prediction

For our task, we should create an annotate result from some video. With this, we should use a video of road traffic, which we then annotate with our trained model, image by image, comparing different trained models.

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["image1.jpg", "image2.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk